## Introducing LangGraph

Now that we understand how to interact with foundation models using Amazon Bedrock's Converse API, we'll explore how to build more structured, multi-step AI applications using LangGraph.

### What is LangGraph?

LangGraph is a framework for creating stateful, multi-step workflows with large language models. While the Converse API handles basic back-and-forth interactions, LangGraph allows you to:

- Build directed graphs of conversational states
- Implement conditional logic and decision-making
- Coordinate multiple AI and non-AI components

### Why Use LangGraph?
No reason other than it's helpful to get started. It's a tool in your toolbelt. There are other frameworks out there, but LangGraph is the most common to date. In later sections we'll discuss how to mix and match these frameworks using abstraction in your code.

In the next notebook, we'll dive into building our first LangGraph application, where we'll explore these concepts hands-on. You'll see how these structured approaches can dramatically improve the reliability and capabilities of your AI applications.

Let's get started!

Now let's import our dependencies:

In [ ]:
import boto3
import json

import boto3
import json

REGION = 'us-west-2'

# Initialize the Bedrock client
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

print("✅ Setup complete!")

## Creating a Simple Conversation Graph

Let's create a basic conversation flow that can:
1. Take user input
2. Process it with our LLM
3. Return a response

We'll build this step by step:

In [ ]:
from typing import Dict, List, TypedDict
from langgraph.graph import START, END, Graph

# Define our state type - this maintains conversation context
class State(TypedDict):
    """Type definition for our conversation state"""
    messages: List[Dict[str, any]]  # History of all messages
    current_message: str                # The latest user message

# Create our LLM call function
def call_llm(state: State) -> State:
    """Call Bedrock with the current conversation state"""
    
    # Configure inference parameters
    inference_config: Dict[str, any] = {
        "maxTokens": 2000,    # Maximum tokens to generate
        "temperature": 0      # 0 = deterministic(ish), higher = more creative
    }
    
    # Make the API call to Bedrock
    response = bedrock.converse(
        modelId="anthropic.claude-3-5-haiku-20241022-v1:0",
        messages=state["messages"],
        inferenceConfig=inference_config
    )
    
    # Add the response to our message history
    state["messages"].append(response['output']['message'])
    return state

# Create our graph
workflow = Graph()

# Add our LLM node to the graph
workflow.add_node("llm", call_llm)

# Set up the flow: START -> LLM -> END
workflow.add_edge(START, "llm")  # First, call the LLM
workflow.add_edge("llm", END)    # Then end the workflow

# Compile the graph into an executable chain
chain = workflow.compile()

Now let's try out our conversation graph!

In [ ]:
# Define the prompt text using XML tags for better Claude interaction
PROMPT_TEXT = """
Explain what makes a good prompt.

<context>
I am seeking to understand the characteristics and elements that make an effective prompt for language models.
</context>

<question>
What makes a good prompt?
</question>
"""

# Initial state
initial_state = State(
    messages=[{ "role": "user", "content": [ {"text": PROMPT_TEXT } ] } ],
    current_message=""
)

# Run our graph
result: State = chain.invoke(initial_state)

# Print the result
print(json.dumps(result["messages"][-1], indent=2))


Congratulations! You've just created your first LangGraph conversation flow. This is a simple example, but in the next notebooks we'll build on this to create more complex and useful patterns constructed as graphs using LangGraph.

## Exercise

Try modifying the initial message to ask different questions about prompt engineering. What happens if you adjust the temperature parameter? Try values between 0 and 1 and observe how the responses change.